Ce code d'extraction est très largement inspiré du code de Gabriel Preda disponible ici : https://github.com/gabrielpreda/covid-19-tweets/blob/master/covid-19-tweets.ipynb

# Paramètres

In [3]:
# modules nécessaires
import tweepy as tw
import pandas as pd
from tqdm import tqdm

In [2]:
# clés d'identification
consumer_key = "" #(API key)
consumer_secret = "" #API key secret)
access_token = "" #(acces token)
access_token_secret = ""# access token secret

In [3]:
# authentification avec les clés
auth=tw.OAuthHandler(consumer_key,consumer_secret)

# on paramétrise l'API
    # wait_on_rate_limit permet d'afficher une notif si on dépasse le quota autorisé par Twitter
api=tw.API(auth,wait_on_rate_limit=True)

# Query

In [288]:
# on cherche la racine vaccin (autorise aussi les hashtags, les composés de vaccin)
# on ignore les retweets
query="vaccin OR #vaccin -filter:retweets"

# date limite
date_until='2021-01-15'

# dernier id à extraire
last_id=1349600875061387267

#nom du fichier à nourir
path='fichier_extract'
nb_limit=5000

In [289]:
# on lance la requête (soit avec un id_max, soit avec une date max)
tweets=tw.Cursor(api.search,q=query,lang='fr',max_id=last_id).items(nb_limit)
#tweets=tw.Cursor(api.search,q=query,lang='fr',until=date_until).items(nb_limit)

In [290]:
# on intègre les tweets récupérés dans un vecteur
tweets_data=[]
for tweet in tqdm(tweets):
    tweets_data.append(tweet)

2000it [01:31, 21.77it/s]


In [291]:
print('Nombre de tweets récupérés : {}'.format(len(tweets_data)))
print('\n')
print('Chaque tweet est sous une forme complexe avec une partie en JSON sous cette forme : \n')
print(tweets_data[0])

Nombre de tweets récupérés : 2000


Chaque tweet est sous une forme complexe avec une partie en JSON sous cette forme : 

Status(_api=<tweepy.api.API object at 0x11604e590>, _json={'created_at': 'Thu Jan 14 06:14:57 +0000 2021', 'id': 1349600875061387267, 'id_str': '1349600875061387267', 'text': "RT @jacquesderosier: Je ne suis pas anti vaccin mais ma mère a été vaccinée vendredi. Aujourd'hui on vient de m'informer qu'elle a le Covid…", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'jacquesderosier', 'name': 'Jacques derosier', 'id': 132997206, 'id_str': '132997206', 'indices': [3, 19]}], 'urls': []}, 'metadata': {'iso_language_code': 'fr', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3

In [292]:
# petite vérification : on affiche le début des cinq premiers tweets
for tweet in tweets_data[0:5]:
    print(tweet.text)

RT @jacquesderosier: Je ne suis pas anti vaccin mais ma mère a été vaccinée vendredi. Aujourd'hui on vient de m'informer qu'elle a le Covid…
RT @BaufMalou: L'ONU forcée d'admettre que le vaccin financé par la Fondation Gates est à l'origine d'une épidémie de polio en Afrique (21s…
RT @patricklescop: @FrancisBack5 Moi j’ai un bon copain qui habitait rue Louis Pasteur à Brest donc je sais aussi ce qu’est un vaccin....
RT @Ouestcetiti: Le Président à encore une fois ses nerfs ... et encore une fois contre les anciens ...!!!!!
Il les veut comme cobayes pour…
RT @Conspis_en_PLS: Covid-19 : l'efficacité du vaccin est-elle vraiment contredite par la flambée épidémique en Israël ?

Selon certains sa…


In [293]:
# on intègre la donnée de manière structurée dans un dataframe
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_data):
    # on crée une colonne pour récupérer les hashtags
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        # on récupère le texte complet (et non pas tronqué)
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    # on sélectionne les informations pertinentes
    tweets_df = tweets_df.append(pd.DataFrame({'tweet_id':tweet.id,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'tweet_location':tweet.geo,
                                               'hashtags': [hashtags if hashtags else None],
                                               'is_retweet': tweet.retweeted,
                                               'is_reply':tweet.in_reply_to_screen_name,
                                               'is_quote_status':tweet.is_quote_status,
                                               'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified}, index=[0]))

100%|██████████| 2000/2000 [27:48<00:00,  1.20it/s]   


In [297]:
tweets_df.head()

,tweet_id,date,text,tweet_location,hashtags,is_retweet,is_reply,is_quote_status,user_name,user_location,user_description,user_created,user_followers,user_favourites,user_verified
0,1349600875061387267,2021-01-14 06:14:57,RT @jacquesderosier: Je ne suis pas anti vacci...,None,None,False,None,False,Le bon sens #NS #FF,,,2011-07-20 20:56:59,406,4696,False
0,1349600856195411968,2021-01-14 06:14:53,RT @BaufMalou: L'ONU forcée d'admettre que le ...,None,None,False,None,False,ralo2266,Bretagne,retweeter ne veut pas dire approuver#bzh#srfc❤...,2012-02-01 20:22:32,567,25459,False
0,1349600829003636737,2021-01-14 06:14:46,RT @patricklescop: @FrancisBack5 Moi j’ai un b...,None,None,False,None,False,Francis Back,,Un Républicain En Marche ira toujours plus loi...,2018-02-27 17:59:40,7520,85743,False
0,1349600807717646336,2021-01-14 06:14:41,RT @Ouestcetiti: Le Président à encore une foi...,None,None,False,None,False,carcenac germaine,,,2011-07-17 20:24:42,617,27340,False
0,1349600766961582081,2021-01-14 06:14:31,RT @Conspis_en_PLS: Covid-19 : l'efficacité du...,None,None,False,None,False,Paschase Broët,le Monde et l'Humain,VM + RER + IHS = complémentarités compatibles,2012-10-04 19:07:15,50,6230,False


In [298]:
# on exporte les données dans un csv
tweets_df.to_csv(path,index='False')

In [299]:
# on lit le fichier complet
tweets_old_df = pd.read_csv("final_tweets.csv")
print(tweets_old_df.shape)

(105322, 15)


/Users/carolineboudier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (0,5,7,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [300]:
# on fusionne les anciennes et nouvelles données
full_tweets_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {full_tweets_df.shape[0]}")

new tweets: 2000 past tweets: 105322 all tweets: 107322


In [301]:
# on supprime les doublons
full_tweets_df.drop_duplicates(subset = ["tweet_id"], inplace=True)
print(f"all tweets: {full_tweets_df.shape}")

all tweets: (107321, 15)


In [302]:
# on exporte le fichier final
full_tweets_df.to_csv("final_tweets.csv", index=False)